In [ ]:
# Notes from where data is taken
# CIFAR-10 - https://www.kaggle.com/competitions/cifar-10/data
# MNIST - https://www.kaggle.com/datasets/hojjatk/mnist-dataset
# GTSRB (road signs) - https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/
# Imagenette (10 classes from ImageNet) - https://github.com/fastai/imagenette#imagenette-1

In [ ]:
# CIFAR-10 statistics